# PyTorch Tutorial 10 - Dataset Transforms

https://www.youtube.com/watch?v=X_QOZEko5uE&list=PLqnslRFeH2UrcDBWF5mfPGpqQDSta6VK4&index=10

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

In [2]:
class WineDataset(Dataset):
    def __init__(self, transform=None):  # !!! transform=None
        xy = np.loadtxt('wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        
        self.x = xy[:, 1:]
        self.y = xy[:, [0]]
        
        self.transform = transform
        
    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    def __len__(self):
        return self.n_samples

### hereunder we create a class which will perform necessary transformations (convertion from numpy array into tensor)

In [3]:
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [4]:
dataset = WineDataset(transform=ToTensor())  # !!! transform=ToTensor()
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


### let's add one more class and unify the two classes into a composition of transformations

In [5]:
class MulTransform:  # basically it is a function to multiply features
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, sample):
        inputs, target = sample
        inputs *= self.factor
        return inputs, target

In [6]:
# 'torchvision.transforms.Compose' unifies 2 transformations into one set of transformations
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])  # 2 means multiplied by 2
dataset = WineDataset(transform=composed)  # here we pass the composition of transformations
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)  # we have got features multiplied by 2

<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03]) tensor([1.])
